#  Few-shot распознавание сущностей

Для обучения ner-моделей обычно нужно много данных. Но есть способ сократить требование к объёму данных - использовать предобученные языковые модели вроде Берта. Из-за того, что языковая модель даёт хорошие эмбединги, можно завести ner всего с 10 предложений.

Такой подхол называется Few-shot learning. Подробнее с тем, как это работает, мы разберемся, когда будем проходить нейроныые сети. А пока научимся пользоваться готовыми штуками, чтобы решать практические задачи. 

Берт и возможность тренировать few-shot ner модели есть в deep pavlov (http://docs.deeppavlov.ai/en/master/features/models/ner.html)

In [1]:
!pip install deeppavlov

     |████████████████████████████████| 737kB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 163kB 5.6MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 5.5MB/s eta 0:00:01
     |████████████████████████████████| 8.0MB 4.8MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 4.6MB/s eta 0:00:01     |██████████████████▌             | 10.0MB 4.9MB/s eta 0:00:02
     |████████████████████████████████| 2.8MB 5.1MB/s eta 0:00:01
     |████████████████████████████████| 10.1MB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 3.8MB/s eta 0:00:011
     |████████████████████████████████| 317kB 5.1MB/s eta 0:00:01
     |████████████████████████████████| 6.7MB 11.0MB/s eta 0:00:01
     |████████████████████████████████| 51kB 2.1MB/s eta 0:00:011
     |████████████████████████████████| 51kB 3.5MB/s eta 0:00:011
     |████████████████████████████████| 25.2MB 5.3MB/s eta 0:00:01     |███████████████████████████████▏| 24.5MB 5.3MB/s eta 0:00:01
     |

     |████████████████████████████████| 266kB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 296kB 4.3MB/s eta 0:00:01
     |████████████████████████████████| 7.1MB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 399kB 5.1MB/s eta 0:00:01
     |████████████████████████████████| 163kB 5.6MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.2.5-cp36-none-any.whl size=1392927 sha256=7508bbe2624b8a864d22247016e97f60d23b883b6032ef66d2ad30650c27663b
  Stored in directory: /root/.cache/pip/wheels/da/8c/38/a8a36581975f8d03275c49960019f955e4d19fd14ae7e42d3d
  Created wheel for uvicorn: filename=uvicorn-0.9.0-cp36-none-any.whl size=38247 sha256=1f44dc4e1be507021e3fa40b8093f38607dd380e9bfdcc5705b5aeea01775afc
  Stored in directory: /root/.cache/pip/wheels/19/e5/d1/a50d405d3bb18fac538ef9606ed9b6cd5efb6e06b6de834507
  Created wheel for overrides: filename=overrides-1.9-cp36-none-any.whl size=5046 sha256=8f77bdbdf41b538a81f1505dfdd72c2158ed62c9ecdc75e686dfcf1a935401e0


In [2]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 37kB/s  eta 0:00:01     |██████████▌                     | 135.1MB 4.5MB/s eta 0:01:02     |████████████                    | 155.3MB 5.2MB/s eta 0:00:50     |██████████████████████▏         | 285.2MB 4.7MB/s eta 0:00:27
     |████████████████████████████████| 512kB 4.5MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 5.4MB/s eta 0:00:01     |████████████████████████████    | 3.3MB 5.4MB/s eta 0:00:01


In [7]:
!apt-get --yes install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  git-man libcurl3-gnutls liberror-perl
Suggested packages:
  gettext-base git-daemon-run | git-daemon-sysvinit git-doc git-el git-email
  git-gui gitk gitweb git-cvs git-mediawiki git-svn
The following NEW packages will be installed:
  git git-man libcurl3-gnutls liberror-perl
0 upgraded, 4 newly installed, 0 to remove and 42 not upgraded.
Need to get 4950 kB of archives.
After this operation, 34.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libcurl3-gnutls amd64 7.58.0-2ubuntu3.8 [213 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 liberror-perl all 0.17025-1 [22.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 git-man all 1:2.17.1-1ubuntu0.5 [803 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 git amd64 1:2.17.1-1ubuntu

In [8]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-zhp57c9m
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-zhp57c9m
  Running command git checkout -b feat/multi_gpu --track origin/feat/multi_gpu
  Switched to a new branch 'feat/multi_gpu'
  Branch 'feat/multi_gpu' set up to track remote branch 'feat/multi_gpu' from 'origin'.
  Created wheel for bert-dp: filename=bert_dp-1.0-cp36-none-any.whl size=20140 sha256=99ccc544941d70a8886a626f6ac5deb2aa51bc2add1739866b142d4b046b9bcb
  Stored in directory: /tmp/pip-ephem-wheel-cache-c1c9539p/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [0]:
# загрузка файлов для колаба
# from google.colab import files
# uploaded = files.upload()

Saving valid.txt to valid.txt


In [10]:
# в колабе файлы не перезаписываются, создается новый с (1,2,3..). 
# Чтобы удалить старый файл перед загрузкой нового используйте rm file
# rm .txt удаляет все txt файлы в текущей папке
#!rm *.txt

In [6]:
import pandas as pd

Возьмем тексты с авито по категории приставки. 

In [7]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

Чтобы научиться выделять названия разметим небольшое количество текстов. Для DeepPavlov разметку нужно поделить на три файла: train.txt, valid.txt, test.txt.

In [3]:
!ls *.txt

test.txt  train.txt  valid.txt


Файлы для обучения, валидации и тестирования в формате IOB. B маркирует начало сущности, а I продолжение. O - обозначает несущность. PRS - это сокращение от PRISTAVKA. Можно было и полностью написать. 

In [4]:
!head train.txt

Продам O
PS3 B-PRS
, O
2 O
беспроводных O
джойстика O
в O
комплекте O

Продам O


Попробуем теперь обучить разметчик сущностей поверх многоязычного берта. Знать, что это такое и как работает не обязательно. Обязательно указать путь к обучающий файлам.

In [1]:
import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

ner_model = train_model(ner_config, download=True)

2019-12-19 12:08:47.177 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2019-12-19 12:08:47.185 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...


[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
2019-12-19 12:08:55.232 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/tag.dict]


Using TensorFlow backend.






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use standard

...

INFO:tensorflow:Restoring parameters from /root/model


2019-12-19 12:12:40.976 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 50 tokens with 5 phrases; found: 4 phrases; correct: 0.

precision:  100.00%; recall:  80.00%; FB1:  88.89

	PRS: precision:  100.00%; recall:  80.00%; F1:  88.89 4


2019-12-19 12:12:41.87 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 51 tokens with 5 phrases; found: 5 phrases; correct: 0.

precision:  80.00%; recall:  80.00%; FB1:  80.00

	PRS: precision:  80.00%; recall:  80.00%; F1:  80.00 5




{"valid": {"eval_examples_count": 5, "metrics": {"ner_f1": 88.8889, "ner_token_f1": 90.0}, "time_spent": "0:00:02"}}
{"test": {"eval_examples_count": 5, "metrics": {"ner_f1": 80.0, "ner_token_f1": 93.3333}, "time_spent": "0:00:01"}}


2019-12-19 12:12:41.840 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/tag.dict]
2019-12-19 12:13:08.243 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/model]


INFO:tensorflow:Restoring parameters from /root/model


Обучается оно около 10 минут. Добавление примеров в обучение увеличит время обучения. 

Посмотрим как это все размечается.

In [11]:
ner_model(['Playstation 4', 'Xbox 360 продам', 'Продам ПС 3'])

[[['Playstation', '4'], ['Xbox', '360', 'продам'], ['Продам', 'ПС', '3']],
 [['B-PRS', 'I-PRS'], ['B-PRS', 'I-PRS', 'O'], ['O', 'B-PRS', 'I-PRS']]]

In [8]:
marked = []

for text in data.text.values[:1000]:
    # BERT имеет лимит на длину текста в 512 слов, возьмем даже еще меньше
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    # достанем только тексты с сущностями
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [9]:
marked

[[('Xbox', 'B-PRS'),
  ('в', 'O'),
  ('отличном', 'O'),
  ('состоянии', 'O'),
  (',', 'O'),
  ('2', 'O'),
  ('диска', 'O'),
  ('с', 'O'),
  ('играми', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('Поддержка', 'O'),
  ('носителей', 'O'),
  (':', 'O'),
  ('CD', 'O'),
  (',', 'O'),
  ('DVD', 'O'),
  (',', 'O'),
  ('флэшка', 'O'),
  ('.', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('Wi', 'B-PRS'),
  ('-', 'I-PRS'),
  ('Fi', 'I-PRS'),
  ('(', 'O'),
  ('802', 'O'),
  ('.', 'O'),
  ('11', 'O'),
  (')', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('Bluetooth', 'O'),
  ('.', 'O'),
  ('Цвет', 'O'),
  (':', 'O'),
  ('черный', 'O')],
 [('Sony', 'B-PRS'),
  ('PS3', 'B-PRS'),
  ('в', 'O'),
  ('отличном', 'O'),
  ('состоянии', 'O'),
  (',', 'O'),
  ('продаю', 'O'),
  ('за', 'O'),
  ('ненадобностью', 'O'),
  (',', 'O'),
  ('память', 'O'),
  ('12gb', 'O'),
  ('-', 'O'),
  ('встроенные', 'O'),
  ('+', 'O'),
  ('жесткий', 'O'),
  ('диск', 'O'),
  ('на', 'O'),
  ('160

Приставки конечно размечаются, но ещё и другие слова на английском начали захватываться. Можно попробовать добавить в данные примеры с ними, или сделать даже ещё пару типов сущностей, чтобы модель научилась их разделять.

В этой же моделе есть несколько стандартных сущностей.

In [13]:
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

2019-12-19 12:25:41.729 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz to /root/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz
100%|██████████| 1.32G/1.32G [07:07<00:00, 3.09MB/s]
2019-12-19 12:32:48.739 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz archive into /root/.deeppavlov/models
2019-12-19 12:33:02.850 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2019-12-19 12:33:03.371 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]
2019-12-19 12:33:28.634 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ontonotes

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model


In [14]:
ner_model(['Передай привет Михаилу Нефедову'])

[[['Передай', 'привет', 'Михаилу', 'Нефедову']],
 [['O', 'O', 'B-PERSON', 'I-PERSON']]]

In [16]:
ner_model(['Поезжай в Москву'])

[[['Поезжай', 'в', 'Москву']], [['O', 'O', 'B-GPE']]]

In [18]:
ner_model(['Поезжай в Альпы'])

[[['Поезжай', 'в', 'Альпы']], [['O', 'O', 'B-LOC']]]

In [19]:
ner_model(['Я говорю на русском и английском языках'])

[[['Я', 'говорю', 'на', 'русском', 'и', 'английском', 'языках']],
 [['O', 'O', 'O', 'B-LANGUAGE', 'O', 'B-LANGUAGE', 'O']]]

In [20]:
ner_model(['Бутылка воды стоит 50 рублей'])

[[['Бутылка', 'воды', 'стоит', '50', 'рублей']],
 [['O', 'O', 'O', 'B-MONEY', 'I-MONEY']]]

Но он конечно не идеальный.

In [40]:
ner_model(['ле Биг мак'])

[[['ле', 'Биг', 'мак']], [['B-PERSON', 'I-PERSON', 'I-PERSON']]]